## Gaming using MiniMax Algorithm

In [ ]:
import numpy as np

class CrosswordMinimax:
    def __init__(self, grid, words):
        self.grid = grid
        self.words = words
        self.scores = {"Player 1": 0, "Player 2": 0}

    def is_valid_placement(self, row, col, word, direction):
        """Check if the word can be placed at the given position."""
        if direction == 'right':
            if col + len(word) > len(self.grid[0]):
                return False
            return all(self.grid[row][col + i] in ['#', word[i]] for i in range(len(word)))
        elif direction == 'down':
            if row + len(word) > len(self.grid):
                return False
            return all(self.grid[row + i][col] in ['#', word[i]] for i in range(len(word)))
        return False

    def place_word(self, grid, row, col, word, direction):
        """Place the word in the grid."""
        new_grid = [list(row) for row in grid]
        if direction == 'right':
            for i in range(len(word)):
                new_grid[row][col + i] = word[i]
        elif direction == 'down':
            for i in range(len(word)):
                new_grid[row + i][col] = word[i]
        return np.array(new_grid)

    def heuristic_evaluation(self, scores):
        """Custom heuristic function to evaluate game state."""
        return scores["Player 1"] + scores["Player 2"]  # Sum of both players' scores

    def find_best_solutions(self, depth):
        """Find two distinct best solutions using Minimax with Alpha-Beta pruning."""
        solutions = []
        for _ in range(10):
            words_dict = dict(self.words)
            score, grid, scores, moves = self.minimax(self.grid, words_dict, self.scores.copy(), depth, True, [], float('-inf'), float('inf'))
            solutions.append((score, grid, scores, moves))

        solutions.sort(key=lambda x: x[0], reverse=True)
        return solutions[:2]

    def minimax(self, grid, words, scores, depth, is_maximizing, move_sequence, alpha, beta):
        """Minimax with Alpha-Beta Pruning."""
        if depth == 0 or not words:
            return self.heuristic_evaluation(scores), grid, scores, move_sequence

        best_score = float('-inf') if is_maximizing else float('inf')
        best_grid = None
        best_scores = scores.copy()
        best_moves = []

        for (row, col, direction), word in words.items():
            is_valid = self.is_valid_placement(row, col, word, direction)
            print(f"{('Player 1' if is_maximizing else 'Player 2')}'s Turn: Trying to place '{word}' at ({row}, {col}) {direction}")
            print(f"Remaining Words: {list(words.keys())}")
            print(f"is_valid_placement result: {is_valid}\n")

            if is_valid:
                new_grid = self.place_word(grid, row, col, word, direction)
                new_scores = scores.copy()
                current_player = "Player 1" if is_maximizing else "Player 2"
                new_scores[current_player] += len(word)  # Ensure score is updated correctly
                new_words = words.copy()
                del new_words[(row, col, direction)]

                new_move_sequence = move_sequence + [(word, (row, col), direction)]
                score, _, updated_scores, moves = self.minimax(new_grid, new_words, new_scores, depth - 1, not is_maximizing, new_move_sequence, alpha, beta)

                if is_maximizing:
                    if score > best_score:
                        best_score, best_grid, best_scores, best_moves = score, new_grid, updated_scores.copy(), moves
                    alpha = max(alpha, best_score)
                else:
                    if score < best_score:
                        best_score, best_grid, best_scores, best_moves = score, new_grid, updated_scores.copy(), moves
                    beta = min(beta, best_score)

                if beta <= alpha:
                    break  # Alpha-Beta Pruning

        if best_grid is None:
            return self.heuristic_evaluation(scores), grid, scores, move_sequence

        return best_score, best_grid, best_scores, best_moves

# Initial Grid
initial_grid = np.array([
    ['#', '#', '#', '#', 'R', '#', '#', '#', '#'],
    ['#', '#', '#', '#', 'A', '#', '#', '#', '#'],
    ['#', '#', '#', '#', 'B', '#', '#', '#', '#'],
    ['#', '#', '#', '#', 'B', '#', '#', '#', '#'],
    ['H', '#', '#', '#', 'I', '#', '#', '#', '#'],
    ['O', '#', 'C', 'A', 'T', '#', '#', '#', '#'],
    ['R', '#', 'A', '#', '#', '#', 'D', 'O', 'G'],
    ['S', '#', 'M', '#', '#', '#', 'O', '#', '#'],
    ['E', 'L', 'E', 'P', 'H', 'A', 'N', 'T', '#'],
    ['#', '#', 'L', '#', '#', '#', 'K', '#', '#'],
    ['#', '#', '#', '#', '#', '#', 'E', '#', '#'],
    ['#', 'M', 'O', 'N', 'K', 'E', 'Y', '#', '#']
])

words = {
    (5, 2, 'right'): 'CAT',
    (6, 6, 'right'): 'DOG',
    (8, 1, 'right'): 'ELEPHANT',
    (11, 1, 'right'): 'MONKEY',
    (4, 0, 'down'): 'HORSE',
    (0, 4, 'right'): 'RABBIT',
    (2, 5, 'down'): 'CAMEL',
    (6, 6, 'down'): 'DONKEY'
}

crossword_game = CrosswordMinimax(initial_grid, words)

depth = len(words)
best_solutions = crossword_game.find_best_solutions(depth)

for i, (score, grid, scores, moves) in enumerate(best_solutions, 1):
    print(f"Solution {i} - Score: {score}")
    print("Scores:", scores)
    print("Moves:")
    for move in moves:
        print(move)
    print("Final Grid:")
    for row in grid:
        print(" ".join(row))
    print("\n---------------------\n")

Player 1's Turn: Trying to place 'CAT' at (5, 2) right
Remaining Words: [(5, 2, 'right'), (6, 6, 'right'), (8, 1, 'right'), (11, 1, 'right'), (4, 0, 'down'), (0, 4, 'right'), (2, 5, 'down'), (6, 6, 'down')]
is_valid_placement result: True

Player 2's Turn: Trying to place 'DOG' at (6, 6) right
Remaining Words: [(6, 6, 'right'), (8, 1, 'right'), (11, 1, 'right'), (4, 0, 'down'), (0, 4, 'right'), (2, 5, 'down'), (6, 6, 'down')]
is_valid_placement result: True

Player 1's Turn: Trying to place 'ELEPHANT' at (8, 1) right
Remaining Words: [(8, 1, 'right'), (11, 1, 'right'), (4, 0, 'down'), (0, 4, 'right'), (2, 5, 'down'), (6, 6, 'down')]
is_valid_placement result: False

Player 1's Turn: Trying to place 'MONKEY' at (11, 1) right
Remaining Words: [(8, 1, 'right'), (11, 1, 'right'), (4, 0, 'down'), (0, 4, 'right'), (2, 5, 'down'), (6, 6, 'down')]
is_valid_placement result: True

Player 2's Turn: Trying to place 'ELEPHANT' at (8, 1) right
Remaining Words: [(8, 1, 'right'), (4, 0, 'down'), (0, 